In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 5.5MB/s 
     |████████████████████████████████| 3.0MB 27kB/s 
     |████████████████████████████████| 1.1MB 55.5MB/s 
     |████████████████████████████████| 890kB 57.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2a7826c1a77bd1e64127153222be5b8f721e512b1bed832b77146e263655f16e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [98]:
import transformers


MAX_LEN = 150
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 10
BASE_MODEL_PATH = 'bert-base-chinese'
MODEL_PATH = 'model.bin'
pre_TRAINING_FILE = '/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/train/new_train.csv'
TRAINING_FILE = '/content/train_dataset.csv'
TOKENIZER = transformers.BertTokenizer.from_pretrained(
    BASE_MODEL_PATH,
    do_lower_case=True
)


In [99]:
import pandas as pd
import numpy as np
import torch
import string


import torch
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import numpy as np

# from model import BERTBaseUncased
from sklearn import model_selection
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from collections import Counter 

from sklearn.metrics import accuracy_score, roc_curve, auc

In [100]:
# ddd = pd.read_csv(pre_TRAINING_FILE)


In [101]:
# def search(pattern, sequence):
#     """从sequence中寻找子串pattern
#     如果找到，返回第一个下标；否则返回-1。
#     """
#     n = len(pattern)
#     for i in range(len(sequence)):
#         if sequence[i:i + n] == pattern:
#             return i
#     return -1

In [102]:
# def search_duplicate_trigger(pattern, sequence):
#     """从sequence中寻找子串pattern
#     如果找到，返回第一个下标；否则返回-1。
#     """
#     n = len(pattern)
#     temp = []
#     for i in range(len(sequence)):
#         if sequence[i:i + n] == pattern:
#             temp.append(i)
#     if len(temp) == 1:
#         return 0,temp
#     elif len(temp) > 1:
#         return 1,temp
#     else:
#         return -1,temp

In [103]:

# def new_trigger(df):
#     triggers = []
#     questionable_triggers_mark = []
#     questionable_triggers_list = []
#     for idx, row in df.iterrows():
#         loc = search(row['trigger'], row['news'])
#         question_mark, temp = search_duplicate_trigger(row['trigger'], row['news'])
#         triggers.append(str(loc) + row['trigger'])
#         questionable_triggers_mark.append(question_mark)
#         questionable_triggers_list.append(temp)
#     df['trigger'] = triggers
#     df['QuestionMark'] = questionable_triggers_mark
#     df['QuestionLabels'] = questionable_triggers_list
#     return df

In [106]:
df = pd.read_csv(pre_TRAINING_FILE)

In [107]:
df.head()

,Unnamed: 0,id,news,trigger,object,subject,time,location
0,0,0,新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...,启动,人力资源社会保障部,第七届大中城市联合招聘高校毕业生春季专场活动,7日,NaN
1,1,1,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,发布,活动,一批适合高校毕业生的就业岗位,NaN,NaN
2,2,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,鼓励,活动,供需双方,NaN,NaN
3,3,3,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,举行,三大运营商中国移动、中国电信和中国联通,线上发布会,4月8日,NaN
4,4,4,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,发布,三大运营商中国移动、中国电信和中国联通,5G消息白皮书,4月8日,NaN


In [120]:

# entity_type = ['trigger','object','subject','time','location']
entity_type = ['trigger','time']
new_df = pd.DataFrame(columns=['id', 'news', 'question', 'answer'])
for i, row in df.iterrows():
    for title in entity_type:
        if str(row[title]) == row[title]:
            print(row['trigger'])
        break
    break


启动


In [121]:
def data_prepare(df):
    # entity_type = ['trigger','object','subject','time','location']
    entity_type = ['trigger','time']
    new_df = pd.DataFrame(columns=['id', 'news', 'question', 'answer'])
    
    for i, row in df.iterrows():
        for title in entity_type:
            if str(row[title]) == row[title] and title != 'trigger':
                question = row['trigger'] + title
                answer = row[title]
                new_df = new_df.append(pd.DataFrame([[row['id'], row['news'], question, answer]], columns=['id', 'news', 'question', 'answer']))
            elif str(row[title]) == row[title] and title == 'trigger':
                question = title
                answer = row[title]
                new_df = new_df.append(pd.DataFrame([[row['id'], row['news'], question, answer]], columns=['id', 'news', 'question', 'answer']))
    new_df = new_df.reset_index(drop=True)
    new_df.head()             
    return new_df   

In [136]:
class QADataset:
    def __init__(self, id, news, trigger, question, answer):#
        self.id = id
        self.news = news
        self.trigger = trigger
        self.question = question
        self.answer = answer
        self.max_len = MAX_LEN
        self.tokenizer = TOKENIZER
    def __len__(self):
        return len(self.news)
    def __getitem__(self, item):
        id = self.id[item]
        news = self.news[item]
        # trigger = self.trigger[item]
        question = self.question[item]
        answer = self.answer[item]
        encoding = TOKENIZER.encode_plus(
            # trigger,
            question,
            news,
            max_length = self.max_len,
            add_special_tokens = True,
            truncation = True,
            padding=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_token_type_ids=True
            )
        idx0 = -1
        idx1 = -1
        sel_answer = TOKENIZER.encode(answer)[1:-1]
        len_answer = len(sel_answer)
        for ind in (i for i, e in enumerate(encoding['input_ids']) if e == sel_answer[0]):
            if encoding['input_ids'][ind: ind + len_answer] == sel_answer:
                idx0 = ind
                idx1 = ind + len_answer -1
                break
        targets_start = [0] * 150
        targets_end = [0] * 150
        targets_start[idx0] = 1
        targets_end[idx1] = 1
        padding_length = self.max_len - len(encoding['input_ids'])
        input_ids = encoding['input_ids'] + ([0] * padding_length)
        attention_mask = encoding['attention_mask'] + ([0] * padding_length)
        token_type_ids = encoding['token_type_ids'] + ([0] * padding_length)   
        
        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "targets_start": torch.tensor(targets_start, dtype=torch.long),
            "targets_end": torch.tensor(targets_end, dtype=torch.long),
            "padding_length": torch.tensor(padding_length, dtype=torch.long),
            "id": id,
            "news": news,
            "question": question,
            "answer": answer
        }

In [123]:
class QAModel(nn.Module):
    def __init__(self):
        super(QAModel, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BASE_MODEL_PATH)
        self.drop_out = nn.Dropout(0.3)
        self.dense = nn.Linear(768, 2)
        torch.nn.init.normal_(self.dense.weight, std=0.02)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        # sequence_output: (batch_size, num_tokens, 768)
        # logits: (batch_size, num_tokens, 2)
        # start_logits, end_logits: (batch_size, num_tokens, 1), (batch_size, num_tokens, 1)
        # After squeeze: (batch_size, num_tokens), (batch_size, num_tokens)
        sequence_output, pooled_output = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        logits = self.dense(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)

        return start_logits, end_logits

In [124]:
def loss_fn(o1, o2, t1, t2):
    # print(o1[0])
    # print(t1)
    l1 = nn.BCEWithLogitsLoss()(o1, t1)
    l2 = nn.BCEWithLogitsLoss()(o2, t2)
    return l1 + l2

class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [125]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    losses = AverageMeter()
    # tk0 = tqdm(data_loader, total=len(data_loader))
    # tqdm(data_loader, total=len(data_loader)).close()
    try:
        with tqdm(data_loader, total=len(data_loader)) as tk0:
            for bi, d in enumerate(tk0):
                input_ids = d['input_ids']
                token_type_ids = d['token_type_ids']
                attention_mask = d['attention_mask']
                targets_start = d['targets_start']
                targets_end = d['targets_end']
                question = d['question']
                news = d['news']
                answer = d['answer']
                
                input_ids = input_ids.to(device, dtype=torch.long)
                token_type_ids = token_type_ids.to(device, dtype=torch.long)
                attention_mask = attention_mask.to(device, dtype=torch.long)
                targets_start = targets_start.to(device, dtype=torch.float)
                targets_end = targets_end.to(device, dtype=torch.float)

                optimizer.zero_grad()
                outputs_start, outputs_end = model(
                    input_ids= input_ids,
                    attention_mask = attention_mask,
                    token_type_ids = token_type_ids
                )
        #         print(outputs_start)
        #         print(o2)
        #         outputs_start = torch.softmax(outputs_start, dim=1)
        #         outputs_end = torch.softmax(outputs_end, dim=1)
        #         outputs_start=torch.argmax(outputs_start,dim=1)
        #         outputs_end=torch.argmax(outputs_end,dim=1)
                loss = loss_fn(outputs_start, outputs_end, targets_start, targets_end)
                loss.backward()
                optimizer.step()
                scheduler.step()
                losses.update(loss.item(), input_ids.size(0))
                tk0.set_postfix(loss = losses.avg)
    except KeyboardInterrupt:
        tk0.close()
        raise
    tk0.close()

        

In [126]:
def eval_fn(data_loader, model, device):
    model.eval()
    # losses = AverageMeter()
    pre_starts = []
    pre_ends = []
    real_starts = []
    real_ends = []
    pred_text = []
    real_text = []
    # tqdm(data_loader, total=len(data_loader)).close()
    try:
        with tqdm(data_loader, total=len(data_loader)) as tk0:
            for bi, d in enumerate(tk0):
                input_ids = d['input_ids']
                # print(input_ids)
                token_type_ids = d['token_type_ids']
                attention_mask = d['attention_mask']
                targets_start = d['targets_start']
                targets_end = d['targets_end']
                question = d['question']
                news = d['news']
                # print(news)
                answer = d['answer']
                # print(answer)
                
                input_ids = input_ids.to(device, dtype=torch.long)
                token_type_ids = token_type_ids.to(device, dtype=torch.long)
                attention_mask = attention_mask.to(device, dtype=torch.long)
                targets_start = targets_start.to(device, dtype=torch.float)
                # print(f"targets_start: {targets_start}")

                targets_end = targets_end.to(device, dtype=torch.float)
                # print(f"targets_end: {targets_end}")

                o1, o2 = model(
                    input_ids= input_ids,
                    attention_mask = attention_mask,
                    token_type_ids = token_type_ids
                )
                y = torch.softmax(o1, dim=1)
                outputs_start = torch.zeros_like(y)
                y = torch.argmax(y,dim=1)
                for i, t in enumerate(outputs_start):
                    outputs_start[i][y[i]] = 1
                # print(f"outputs_start: {outputs_start}")
                y = torch.softmax(o2, dim=1)
                outputs_end = torch.zeros_like(y)
                y = torch.argmax(y,dim=1)
                for i, t in enumerate(outputs_end):
                    outputs_end[i][y[i]] = 1
                # print(f"outputs_end: {outputs_end}")
                # torch.mul(input_ids, outputs_start)
                for i, j, z in zip(outputs_start,outputs_end, input_ids):
                    text_ids = z.cpu().numpy()
                    start_index = torch.where(i==1)[0].cpu().numpy()[0]
                    end_index = torch.where(j==1)[0].cpu().numpy()[0]
                    aa = text_ids[start_index:end_index+1]
                    temp = "".join(TOKENIZER.ids_to_tokens[i] for i in aa)
                    temp = temp.replace('#','')
                    temp = temp.replace('[UNK]','')
                    pred_text.append(temp)
                    

        

                # pred_text.extend()
                # real_text.extend(answer)
                # pre_starts.extend(outputs_start.cpu().detach().numpy().tolist())
                # pre_ends.extend(outputs_end.cpu().detach().numpy().tolist())
                # real_starts.extend(targets_start.cpu().detach().numpy().tolist())
                # real_ends.extend(targets_end.cpu().detach().numpy().tolist())
                real_text.extend(answer)

                
            # return input_ids, news, answer, pre_starts, pre_ends, real_starts, real_ends, 
            return real_text, pred_text

    except KeyboardInterrupt:
        tk0.close()
        raise
    tk0.close()

In [127]:
dfx = pd.read_csv(pre_TRAINING_FILE)
dfx = dfx[dfx['trigger'].isna()==False]
dfx1 = dfx.loc[:8000]
dfx2 = dfx.loc[8000:]

In [128]:
dfx1 = data_prepare(dfx1)
dfx2 = data_prepare(dfx2)
dfx1.drop_duplicates(inplace=True)
dfx2.drop_duplicates(inplace=True)
dfx1.head()

,id,news,question,answer
0,0,新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...,trigger,启动
1,0,新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...,启动time,7日
2,1,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,trigger,发布
3,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,trigger,鼓励
4,3,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,trigger,举行


In [129]:
dfx2.tail()

,id,news,question,answer
10750,15997,星岛环球网消息：中新网4月8日电据广州市公安局官方微博消息，银川白云警方4月8日报信：经过缜...,报信time,4月8日
10751,15998,4月8日零时起，深航除死灰复燃武汉往返深圳的航班外，还同时死灰复燃了武汉来去昆明、塞阿拉州、...,trigger,死灰复燃
10752,15998,4月8日零时起，深航除死灰复燃武汉往返深圳的航班外，还同时死灰复燃了武汉来去昆明、塞阿拉州、...,死灰复燃time,4月8日零时起
10753,15999,张女士告诉记者，1月22日，张女士带着父母和丫头来深圳过年，谁知一待就是70多天,trigger,来
10754,15999,张女士告诉记者，1月22日，张女士带着父母和丫头来深圳过年，谁知一待就是70多天,来time,1月22日


In [137]:
# dfx = pd.read_csv(TRAINING_FILE).dropna().reset_index(drop=True)
# dfx = pd.read_csv(TRAINING_FILE)
# dfx = data_prepare(dfx)
df_train, df_valid = model_selection.train_test_split(
    dfx1, test_size=0.2#, random_state=42#, stratify=dfx.question.values
)
df_train = pd.concat([df_train,dfx2])
df_train = df_train.reset_index(drop=True)

df_valid = df_valid.reset_index(drop=True)

train_dataset = QADataset(
    id = df_train.id.values,
    news=df_train.news.values,
    question=df_train.question.values,
    answer=df_train.answer.values
)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4
)
valid_dataset = QADataset(
    id=df_valid.id.values,
    news=df_valid.news.values,
    question=df_valid.question.values,
    answer=df_valid.answer.values
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1
)
# device = torch.device("cuda")
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')
model = QAModel()
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {
        "params": [
            p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.001,
    },
    {
        "params": [
            p for n, p in param_optimizer if any(nd in n for nd in no_decay)
        ],
        "weight_decay": 0.0,
    },
]
num_train_steps = int(len(df_train) / TRAIN_BATCH_SIZE * EPOCHS)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_train_steps
)

model = nn.DataParallel(model)


In [138]:
def f1_cal(real_text, pred_text):
    PspanTotal, RspanTotal, PspanSingle = 1e-10, 1e-10, 1e-10
    for R, P in zip(real_text,pred_text):
        if str(P) == P:
            PspanTotal += 1 
        if str(R) == R:
            RspanTotal += 1
            PspanSingle += 1 if R == P else 0
            
    Pspan = PspanSingle / PspanTotal
    Rspan = PspanSingle / RspanTotal
    F1span = (2 * Pspan * Rspan) / (Pspan + Rspan)

    PcharTotal, RcharTotal, PcharSingle = 1e-10, 1e-10, 1e-10
    for R, P in zip(real_text,pred_text):
        if str(P) == P:
            PcharTotal += len(str(P)) 
        if str(R) == R:
            RcharTotal += len(str(R))
            x = Counter(str(R))
            y = Counter(str(P))
            c = x&y
            PcharSingle += sum(c.values())
            
            
    Pchar = PcharSingle / PcharTotal
    Rchar = PcharSingle / RcharTotal
    F1char = (2 * Pchar * Rchar) /( Pchar + Rchar)

    return (F1span + F1char) / 2
            

# print(f1_cal(real_text, pred_text))
    # R = set([SPOLT(spolt, self.data.tokenizer) for spolt in R])
    # T = set([SPOLT(spolt, self.data.tokenizer) for spolt in T])
    # X += len(R & T)
    # Y += len(R)
    # Z += len(T)
    # f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z

In [139]:
# best_jaccard = 0
best_f1 = 0
early_stopping = 0
for epoch in range(EPOCHS):
    if early_stopping == 3:
        break
    early_stopping += 1
    train_fn(train_data_loader, model, optimizer, device, scheduler)
    real_text, pred_text = eval_fn(valid_data_loader, model, device)
    # outputs = np.argmax(np.array(outputs),axis=1) # >= 0.5
    # print(outputs)
    # accuracy1 = accuracy_score(starts, targets_start)
    # accuracy2 = accuracy_score(ends, targets_end)
    # accuracy = (accuracy1+accuracy2)/2.0
    f1 = f1_cal(real_text, pred_text)
    print(f"F1 Score = {f1}")
    if f1 > best_f1:
        early_stopping = 0
        torch.save(model.state_dict(), MODEL_PATH)
        torch.save(model, '/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/new2_joint_time_model.pt')
        best_f1 = f1

#     jaccard = eval_fn(valid_data_loader, model, device)
#     print(f"Jaccard Score = {jaccard}")
#     if jaccard > best_jaccard:
#         torch.save(model.state_dict(), MODEL_PATH)
#         best_jaccard = jaccard

    

100%|██████████| 269/269 [00:23<00:00, 11.57it/s]


F1 Score = 0.5565503139095015


  0%|          | 0/605 [00:00<?, ?it/s]

F1 Score = 0.5556867495552511


  1%|          | 4/605 [00:05<12:39,  1.26s/it, loss=0.0209]


KeyboardInterrupt: ignored

In [ ]:
# # meta_data = joblib.load('meta.bin')
# # enc_pos = meta_data['enc_pos']
# # enc_bems = meta_data['enc_bems']
# # enc_tag = meta_data['enc_tag']

# # num_tag = len(list(enc_tag.classes_))
# # num_bems = len(list(enc_bems.classes_))
# # num_pos = len(list(enc_pos.classes_))

# # sentence = testdata.iloc[9]['news']

# # news = testdata.iloc[31]['news']
# news = '4月起地铁全线网开展消防普查广州市消防救援支队副支队长王学强介绍，虽然该起火灾过火面积小、很快就被扑灭，但是也暴露了地铁公司在落实自查自纠、消防培训教育和警示宣传方面还存在不足之处'

# #恢复
# entity_type = ['object', 'subject', 'time', 'location']
# # questions = ['加入'+i for i in entity_type]
# # question = questions[0]
# question = '介绍time'
# test_dataset = QADataset(
#     news=[news],
#     question=[question],
#     answer=[[0] * len(news)]
# )
# test_data_loader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=1, num_workers=0
# )
# # test_dataset = QADataset(
# #     news=testdata.news.values,
# #     question=testdat.question.values,
# #     answer=[[0] * len(news)]
# # )
# device = torch.device('cuda')
# # model = QAModel()
# # model.load_state_dict(torch.load(MODEL_PATH),False)
# model = torch.load('model.pt')
# model.to(device)

# with torch.no_grad():
#     for data in test_data_loader:
#         # print(data)
#         input_ids = data['input_ids']
#         input_ids = input_ids.to(device, dtype=torch.long)
#         attention_mask = data['attention_mask']
#         attention_mask = attention_mask.to(device, dtype=torch.long)
#         # attention_mask.view_as(1,attention_mask.shape[0])
#         token_type_ids = data['token_type_ids']
#         token_type_ids = token_type_ids.to(device, dtype=torch.long)
#         o1, o2 = model(
#                         input_ids= input_ids,
#                         attention_mask = attention_mask,
#                         token_type_ids = token_type_ids
#                     ) 
#         y_start = torch.softmax(o1, dim=1)
#         outputs_start = torch.zeros_like(y_start)
#         y_start = torch.argmax(y_start,dim=1)
#         y_start = y_start - len(TOKENIZER.encode(question))
#         # for i, t in enumerate(outputs_start):
#         #     outputs_start[i][y[i]] = 1

#         y_end = torch.softmax(o2, dim=1)
#         outputs_end = torch.zeros_like(y_end)
#         y_end = torch.argmax(y_end,dim=1)
#         y_end = y_end - len(TOKENIZER.encode(question))
#         # for i, t in enumerate(outputs_end):
#         #     outputs_end[i][y[i]] = 1
#         print(news[y_start:y_end+1])

     
# #     # # data = test_dataset
# #     # for k, v in test_dataset.items():
# #     #     data[k] = v.to(device).unsqueeze(0)
# #     # outputs_start, outputs_end = model(
# #     #                 input_ids= input_ids,
# #     #                 attention_mask = attention_mask,
# #     #                 token_type_ids = token_type_ids
# #     #                 )
    
# #     # print(tag.shape)
# #     # print(tag)
# #     # print(tag.argmax(2).cpu().numpy()[0])
# #     print(enc_tag.inverse_transform(tag.argmax(dim=2).cpu().numpy()[0])[1:len(tokenized_sentence)-1])
# #     print(enc_bems.inverse_transform(bems.argmax(2).cpu().numpy()[0])[1:len(tokenized_sentence)-1])
# #     print(enc_pos.inverse_transform(pos.argmax(2).cpu().numpy()[0])[1:len(tokenized_sentence)-1])
# #     # # print(enc_tag.inverse_transform(tag.argmax(dim=2).cpu().numpy()[0])[1:len(tokenized_sentence)-1]=='B-trigger')
# #     # print(sentence)
# #     tt = enc_tag.inverse_transform(tag.argmax(dim=2).cpu().numpy()[0])[1:len(tokenized_sentence)-1]
# #     # for i, j  in zip(sentence,tt):
# #     #     print(i)
# #     #     print(j)
# #     # print([i for i, j  in zip(sentence,tt) if i in ['B-location', 'B-object', 'B-subject', 'B-time', 'B-trigger', 'I-location','I-object', 'I-subject', 'I-time', 'I-trigger']][1:len(tokenized_sentence)-1])







In [154]:
final

,id,news,trigger+type,entity
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,trigger,恢复
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,trigger,作出
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,trigger,查封
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,trigger,
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,trigger,参与
...,...,...,...,...
1195,9976,第二天，国务院联防机制医疗物资保障组再次给企业发函，要求“自即日起，你公司生产、储备的所有医...,trigger,发函
1196,9978,在此前的一次紧急会议上，长垣市委、市政府召集有关局委办负责人，与42家卫材企业代表开会，解决...,trigger,
1197,9979,”2月2日，长垣市防控指挥部发布1号令，要求各卫材企业生产的所有防护用品要统一收储、调配，任...,trigger,发布
1198,9981,”大广高速长垣进站口，市场监督管局联合警方对出长垣市的车辆进行检查，确保医疗物资不被私自运出市区,trigger,检查


In [157]:
new_df = pd.DataFrame(columns=['id', 'news', 'question'])
for i, row in final.iterrows():
    id = row['id']
    question = row['entity'] + 'time'
    new_df = new_df.append(pd.DataFrame([[id, row['news'], question]], columns=['id', 'news', 'question']))
print(new_df)

      id                                               news question
0   7953  此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...   恢复time
0   7955  同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...   作出time
0   7956  2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...   查封time
0   7959  广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...     time
0   7960    作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设   参与time
..   ...                                                ...      ...
0   9976  第二天，国务院联防机制医疗物资保障组再次给企业发函，要求“自即日起，你公司生产、储备的所有医...   发函time
0   9978  在此前的一次紧急会议上，长垣市委、市政府召集有关局委办负责人，与42家卫材企业代表开会，解决...     time
0   9979  ”2月2日，长垣市防控指挥部发布1号令，要求各卫材企业生产的所有防护用品要统一收储、调配，任...   发布time
0   9981   ”大广高速长垣进站口，市场监督管局联合警方对出长垣市的车辆进行检查，确保医疗物资不被私自运出市区   检查time
0   9983                   自即时起，中国内地门店菜品价格恢复到今年1月26日门店停业前标准   恢复time

[1200 rows x 3 columns]


In [158]:
test_data = new_df.reset_index(drop=True)
test_data.head() 

,id,news,question
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,恢复time
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,作出time
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,查封time
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,time
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,参与time


In [141]:

test_data = pd.read_csv('/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/test/test.csv')
for i, row in test_data.iterrows():
    id = row['id']
    question = 'trigger'
    new_df = new_df.append(pd.DataFrame([[id, row['news'], question]], columns=['id', 'news', 'question']))
test_data = new_df.reset_index(drop=True)
test_data.head() 


,id,news,question,answer
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,trigger,NaN
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,trigger,NaN
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,trigger,NaN
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,trigger,NaN
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,trigger,NaN


In [142]:
# import pickle
# # with open('/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/train/triggers.pickle', 'rb') as handle:
# #     test_data = pickle.load(handle)
# test_data = pd.read_csv('/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/train/test.csv')

# ids = []
# news = []
# triggers = []
# for idx, row in test_data.iterrows():
#     for trigger in row['trigger']:
#         # print(trigger)
#         ids.append(int(row['id']))
#         news.append(row['news'])
#         triggers.append(trigger)
# test_data = pd.DataFrame(zip(ids,news,triggers), columns=['id','news','trigger'])
# entity_type = ['object','subject','time','location']
# new_df = pd.DataFrame(columns=['id', 'news', 'question'])
# for i, row in test_data.iterrows():
#     for title in entity_type:
#         id = row['id']
#         question = row['trigger'] + title
#         new_df = new_df.append(pd.DataFrame([[id, row['news'], question]], columns=['id', 'news', 'question']))
# test_data = new_df.reset_index(drop=True)
# test_data.head() 

In [159]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')

# model = QAModel()
# model.load_state_dict(torch.load('model.bin'))
model = torch.load('/content/gdrive/My Drive/事件抽取挑战赛_事件抽取初赛数据/事件抽取挑战赛_事件抽取初赛数据/new2_joint_time_model.pt')
model.to(device)

DataParallel(
  (module): QAModel(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(21128, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (La

In [160]:

test_dataset = QADataset(
    id = test_data.id.values,
    news=test_data.news.values,
    question=test_data.question.values,
    answer=[[0] * len(i) for i in test_data.news.values]
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, num_workers=0
)

In [161]:
next(iter(test_data_loader))['id'].numpy()[0]

7953

In [162]:
import transformers
import torch
import pandas as pd
import numpy as np
# from model import QAModel
# import config
# import dataset
final_ids = []
final_news = []
final_question =[]
final_answer = []
with torch.no_grad():
    test_data_loader
    for data in test_data_loader:
        input_ids = data['input_ids']
        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = data['attention_mask']
        attention_mask = attention_mask.to(device, dtype=torch.long)
        # attention_mask.view_as(1,attention_mask.shape[0])
        token_type_ids = data['token_type_ids']
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        o1, o2 = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        real_id = data['id'].numpy()[0]
        real_news = data['news'][0]
        tok_news = TOKENIZER.encode(data['news'][0])[1:-1]
        question = data['question'][0]
        y_start = torch.softmax(o1, dim=1)
        outputs_start = torch.zeros_like(y_start)
        y_start = torch.argmax(y_start, dim=1)
        y_start = y_start - len(TOKENIZER.encode(question))
        # for i, t in enumerate(outputs_start):
        #     outputs_start[i][y[i]] = 1

        y_end = torch.softmax(o2, dim=1)
        outputs_end = torch.zeros_like(y_end)
        y_end = torch.argmax(y_end, dim=1)
        y_end = y_end - len(TOKENIZER.encode(question))
        # for i, t in enumerate(outputs_end):
        #     outputs_end[i][y[i]] = 1
        # print(real_news[y_start:y_end + 1])
        final_ids.append(real_id)
        final_news.append(real_news)
        final_question.append(question)
        aa = tok_news[y_start:y_end + 1]
        
        final_answer.append("".join(TOKENIZER.ids_to_tokens[i] for i in aa))
        # print(final_answer)
        


In [163]:
final = pd.DataFrame(zip(final_ids, final_news,final_question,final_answer), columns=['id', 'news', 'trigger+type', 'entity'])

In [164]:
final.head(20)

,id,news,trigger+type,entity
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,恢复time,近日
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,作出time,4月3日
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,查封time,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,time,4月8日9时许
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,参与time,1月29日
5,7962,3月23日，李海建曾告诉南都记者，那时候，他每天都很关注雷神山的病人数量，以及产生的医疗废弃...,欢呼time,3月23日
6,7964,据央视消息，随着患者的陆续减少，雷神山医院继续减少病区,减少，雷神山医院继续减少time,减少
7,7965,1月底来雷神山建设医疗废弃物焚烧车间后，李海建曾委托公司发布志愿者招募信息，公司同事和很多素...,来time,1月底
8,7966,4月7日，李海建告诉南都记者，现在，除公司同事以外的大部分志愿者都已离开雷神山，有的正在隔离...,离开time,4月7日
9,7970,”此外，3月31日至4月3日期间，受武汉市疾病预防控制中心邀请，李海建参与了《关于印发办公场...,参与time,3月31日至4月3日期间


In [149]:
final.drop_duplicates(inplace=True)

In [150]:
final.head(20)

,id,news,trigger+type,entity
0,7953,此外，深航积极恢复湖北其他城市的航班运力，已于近日陆续恢复襄阳往返深圳、惠州、沈阳、珠海，宜...,trigger,恢复
1,7955,同时，因其违反中国法律、法规规定，妨害社会管理，根据《出境入境管理法》第八十一条第一款，越秀...,trigger,作出
2,7956,2008年间，佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合...,trigger,查封
3,7959,广州男子涉嫌偷拍女同学发布淫秽信息，学校发声4月8日9时许，广州警方发布通报，经过缜密调查，...,trigger,
4,7960,作为广东深圳一家节能企业的董事长，从1月29日开始，李海建便来到武汉参与医疗废弃物焚烧车间建设,trigger,参与
5,7962,3月23日，李海建曾告诉南都记者，那时候，他每天都很关注雷神山的病人数量，以及产生的医疗废弃...,trigger,欢呼
6,7964,据央视消息，随着患者的陆续减少，雷神山医院继续减少病区,trigger,减少，雷神山医院继续减少
7,7965,1月底来雷神山建设医疗废弃物焚烧车间后，李海建曾委托公司发布志愿者招募信息，公司同事和很多素...,trigger,来
8,7966,4月7日，李海建告诉南都记者，现在，除公司同事以外的大部分志愿者都已离开雷神山，有的正在隔离...,trigger,离开
9,7970,”此外，3月31日至4月3日期间，受武汉市疾病预防控制中心邀请，李海建参与了《关于印发办公场...,trigger,参与


In [89]:
# with open('final.pickle', 'rb') as handle:
#     ok = pickle.load(handle)

In [90]:
final.to_csv('final_location.csv')

In [ ]:
a_news = '2018年5月12日14点28分01秒，中国发生了8级大地震，导致8万人死亡。'
a_question = '发生object'

In [ ]:

# test_dataset = QADataset(
#     news=[a_news],
#     question=[a_question],
#     answer=[[0] * len(a_news)]
# )
# test_data_loader = torch.utils.data.DataLoader(
#     test_dataset, batch_size=1, num_workers=0
# )

TypeError: ignored

In [ ]:
import transformers
import torch
import pandas as pd
import numpy as np
# from model import QAModel
# import config
# import dataset
final_news = []
final_question =[]
final_answer = []
with torch.no_grad():
    for data in test_data_loader:
        input_ids = data['input_ids']
        input_ids = input_ids.to(device, dtype=torch.long)
        attention_mask = data['attention_mask']
        attention_mask = attention_mask.to(device, dtype=torch.long)
        # attention_mask.view_as(1,attention_mask.shape[0])
        token_type_ids = data['token_type_ids']
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        o1, o2 = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        real_news = data['news'][0]
        tok_news = TOKENIZER.encode(data['news'][0])[1:-1]
        question = data['question'][0]
        y_start = torch.softmax(o1, dim=1)
        outputs_start = torch.zeros_like(y_start)
        y_start = torch.argmax(y_start, dim=1)
        y_start = y_start - len(TOKENIZER.encode(question))
        # for i, t in enumerate(outputs_start):
        #     outputs_start[i][y[i]] = 1

        y_end = torch.softmax(o2, dim=1)
        outputs_end = torch.zeros_like(y_end)
        y_end = torch.argmax(y_end, dim=1)
        y_end = y_end - len(TOKENIZER.encode(question))
        # for i, t in enumerate(outputs_end):
        #     outputs_end[i][y[i]] = 1
        # print(real_news[y_start:y_end + 1])
        final_news.append(real_news)
        final_question.append(question)
        aa = tok_news[y_start:y_end + 1]
        
        final_answer.append("".join(TOKENIZER.ids_to_tokens[i] for i in aa))
        print(final_answer)
        


['深航']
['深航', '湖北其他城市的航班运力']
['深航', '湖北其他城市的航班运力', '深航']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人', '买卖合同纠纷']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人', '买卖合同纠纷', '佛山市南海区某五金机械厂等8名合法债权人']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人', '买卖合同纠纷', '佛山市南海区某五金机械厂等8名合法债权人', '诉前财产保全']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人', '买卖合同纠纷', '佛山市南海区某五金机械厂等8名合法债权人', '诉前财产保全', '佛山市南海区某五金机械厂等8名合法债权人因与罗某丰所经营的某灯具公司发生买卖合同纠纷，于2008年8月向原广州市萝岗区人民法院']
['深航', '湖北其他城市的航班运力', '深航', '湖北其他城市的航班运力', '越秀区公安分局', '限期出境处罚', '佛山市南海区某五金机械厂等8名合法债权人', '买卖合同纠纷', '佛山市南海区某五金机械厂等8名合法债权人', '诉前财产保全', '佛山市南海区某五金机

KeyboardInterrupt: ignored

In [ ]:
df = pd.read_csv(pre_TRAINING_FILE)

In [ ]:
df.head()

,id,news,trigger,object,subject,time,location
0,1,新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生...,启动,人力资源社会保障部,第七届大中城市联合招聘高校毕业生春季专场活动,7日,NaN
1,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,发布,活动,一批适合高校毕业生的就业岗位,NaN,NaN
2,2,活动针对高校毕业生特点和需求，集中归集发布一批适合高校毕业生的就业岗位，并鼓励引导供需双方应...,鼓励,活动,供需双方,NaN,NaN
3,4,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,举行,三大运营商中国移动、中国电信和中国联通,线上发布会,4月8日,NaN
4,4,图集4月8日，三大运营商中国移动、中国电信和中国联通将联合举行线上发布会，发布5G消息白皮书...,发布,三大运营商中国移动、中国电信和中国联通,5G消息白皮书,4月8日,NaN


In [ ]:
import pkuseg

In [ ]:
seg = pkuseg.pkuseg(postag=True)

Downloading: "https://github.com/lancopku/pkuseg-python/releases/download/v0.0.16/postag.zip" to /root/.pkuseg/postag.zip
100%|██████████| 41424981/41424981 [00:01<00:00, 29685428.85it/s]


In [ ]:
df.loc[0]['news']

'新华社北京4月7日电（记者王优玲）人力资源社会保障部7日启动第七届大中城市联合招聘高校毕业生春季专场活动'

In [ ]:
triggers = []
for i in df.iterrows():
    for word, tag in seg.cut(i[1]['news']):
        if tag == 'v':
            triggers.append(word)
    break
print(triggers)

['启动', '联合', '招聘']
